# APA Laboratori 5  - LDA/QDA/NBayes/RegLog      

In [ ]:
# Uncomment to upgrade packages
# !pip install pandas --upgrade --user --quiet
# !pip install numpy --upgrade --user --quiet
# !pip install scipy --upgrade --user --quiet
# !pip install statsmodels --upgrade --user --quiet
# !pip install scikit-learn --upgrade --user --quiet
# !pip install seaborn --upgrade --user --quiet
%load_ext autoreload

In [ ]:
#%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
pd.set_option('precision', 3)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Extra imports
from pandas import read_csv
from sklearn.metrics import confusion_matrix, \
                  classification_report, accuracy_score
from pandas.api.types import CategoricalDtype
from pandas.plotting import scatter_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import LeaveOneOut
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, GaussianNB, CategoricalNB
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from numpy.random import  normal, binomial
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families.family import Binomial
from statsmodels.tools.tools import add_constant
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
def confusion(true, pred, classes):
    """
    Function for pretty printing confusion matrices
    """
    cm =pd.DataFrame(confusion_matrix(true, pred), 
                     index=classes,
                     columns=classes)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    return cm

## Example 1: Visualizing and classifying wines with LDA and QDA

 We have the results of an analysis on wines grown in a region in Italy but derived from three different cultivars.
The analysis determined the quantities of 13 chemical constituents found in each of the three types of wines. 
The goal is to separate the three types of wines:

In [ ]:
wine = read_csv("wine.data", delimiter=',', header=None)
wine_classes = ['cultivar %d'%(i+1) for i in range(3)]
wine.shape
wine.columns = ['Wine_type','Alcohol','Malic_acid','Ash',
                'Alcalinity_of_ash','Magnesium','Total_phenols',
                'Flavanoids','Nonflavanoid_phenols',
                'Proanthocyanins','Color_intensity','Hue',
                'OD280/OD315','Proline']

In [ ]:
wine.Wine_type = wine.Wine_type.astype(CategoricalDtype(categories=[1, 2, 3],  
                                                        ordered=True))
wine.describe(include='all')

In [ ]:
scatter_matrix(wine.loc[:,'Alcohol':'Proline'], 
               alpha=0.2, figsize=(16, 16), 
               diagonal='kde',marker='.',
               c=wine.Wine_type);

### LDA

LDA tries to model the probability $p(y=C_k|X=x)$ by assuming: 
* $p(x|C_k)$ is Gaussian (which means that can be described by $\mu_k$ and $\Sigma_k$)
* All covariance matrix are the same ($\Sigma_k = \Sigma$)

By using bayes formula ($p(A|B) = \frac{P(B|A)P(A)}{P(B)}$) and all these asumptions, we obtain the next discriminant function:

$a_k(x) = x^T\Sigma^{-1}\mu_k - \frac{1}{2}\mu_k^T\Sigma^{-1}\mu_k + log(\pi_k)$

Where $\pi_k$ are the prior probabilities. 

If we call:

$w = \Sigma^{-1}\mu_k$

$w_0=- \frac{1}{2}\mu_k^T\Sigma^{-1}\mu_k + log(\pi_k)$

We obtain a linear representation of the formula.

For this example let's practice a different call mode to lda(), using a formula; this is most useful
 when our data is in a dataframe format: 

In [ ]:
lda_model = LinearDiscriminantAnalysis().fit(wine.loc[:,'Alcohol':'Proline'],
                                             wine.Wine_type)

print('Priors:', lda_model.priors_)
print('Means:\n')
means =pd.DataFrame(lda_model.means_)
means.columns=wine.columns[1:]
means
print('Coefs:')
coefs = pd.DataFrame(lda_model.coef_)
coefs.columns =wine.columns[1:]
coefs.T

print('Intercepts:')
intercepts = pd.DataFrame(lda_model.intercept_)
intercepts

print('Explained Variance Ratio')
pd.DataFrame(lda_model.explained_variance_ratio_ )

We can see that neither Magnesium or Proline seem useful to separate the wines; while
 Flavanoids and Nonflavanoid.phenols do. Ash is mainly used in the LD2.


Here we have an example of how the model is predicting the class of a single sample.

In [ ]:
sample = wine.loc[0,'Alcohol':'Proline']
sample_value = wine.loc[0,'Wine_type']

wine.loc[0,:]

In [ ]:
lda_model.predict(sample.values.reshape(1, -1))

In [ ]:
lda_model.decision_function(sample.values.reshape(1, -1))

In [ ]:
np.matmul(lda_model.coef_, sample) + lda_model.intercept_


Plot the projected data in the first two LDs

 We can see that the discrimination is very good

In [ ]:
wine_trans = lda_model.transform(wine.loc[:,'Alcohol':'Proline'])
fig, ax = plt.subplots(figsize=(8,8))
for i in wine.Wine_type.unique():
    plt.scatter(wine_trans[:,0][wine.Wine_type==i],
                wine_trans[:,1][wine.Wine_type==i],
                label='cultivar %d'%i)
ax.set_xlabel('LD1')
ax.set_ylabel('LD2')
plt.legend();


 If need be, we can add the (projected) means to the plot

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
for i in wine.Wine_type.unique():
    plt.scatter(wine_trans[:,0][wine.Wine_type==i],
                wine_trans[:,1][wine.Wine_type==i],
                label='cultivar %d'%i)
    plt.plot(wine_trans[:,0][wine.Wine_type==i].mean(),
             wine_trans[:,1][wine.Wine_type==i].mean(),
             'k^',markersize=20)
ax.set_xlabel('LD1')
ax.set_ylabel('LD2')
plt.legend();

indeed classification is perfect

In [ ]:
confusion(wine.Wine_type, lda_model.predict(wine.loc[:,'Alcohol':'Proline']), 
          classes=wine_classes)          

Let us switch to leave-one-out cross-validation

In [ ]:
def loocv(X,y,model,classes):
    loo = LeaveOneOut()
    pred=[]
    for train_index, test_index in loo.split(X):
        X_tr, X_ts = X[train_index], X[test_index]
        y_tr, _ = y[train_index], y[test_index]
        model.fit(X_tr,y_tr)
        pred.append(model.predict(X_ts)[0])
    return confusion(y,pred,classes), 1-accuracy_score(y,pred)

In [ ]:
cm, err = loocv(wine.loc[:,'Alcohol':'Proline'].values, 
                wine.Wine_type, 
                LinearDiscriminantAnalysis(),
                wine_classes)

cm

err*100

2 mistakes (on 178 observations): 1.12% error

Quadratic Discriminant Analysis is the same

 problems may arise if for some class there are less (or equal) observations than dimensions
 (is not the case for the wine data)

In [ ]:
qda_model = QuadraticDiscriminantAnalysis().fit(wine.loc[:,'Alcohol':'Proline'],
                                                wine.Wine_type)

print('Priors:\n')
pd.DataFrame(qda_model.priors_)
print('Means:\n')
means =pd.DataFrame(qda_model.means_)
means.columns=wine.columns[1:]
means

 There is no projection this time (because projection is a linear operator and the QDA boundaries are quadratic ones)

 but let's have a look at classification:

In [ ]:
confusion(wine.Wine_type, qda_model.predict(wine.loc[:,'Alcohol':'Proline']), 
          classes=wine_classes)

Let us switch to leave-one-out cross-validation

In [ ]:
cm, err = loocv(wine.loc[:,'Alcohol':'Proline'].values,
                wine.Wine_type, 
                QuadraticDiscriminantAnalysis(),
                wine_classes)

cm

err*100

1 mistake (on 178 observations): 0.56% error

 it would be nice to ascertain which wine is the "stubborn" one: it is a wine of type '2' classified
as class '1'. Maybe there is something special with this wine ...

 In the event of numerical errors (insufficient number of observations per class), we can use regularization.
 
 in this case the regularization parameter (0..1) is applied to the covariance matrix (Sigma) so it is not ill conditioned in this fashion
 
 `(1-reg_param)*Sigma + reg_param*np.eye(n_features)`

### QDA

QDA is very similar to LDA. The main difference is that in this model we do not assume that all the classes have the same covariance. This leads to obtaining a quadratic decision surface.

In [ ]:
qda_model = QuadraticDiscriminantAnalysis(reg_param=0.1).\
                    fit(wine.loc[:,'Alcohol':'Proline'],
                        wine.Wine_type)

print('Priors:', qda_model.priors_)
print('Means:\n')
means =pd.DataFrame(qda_model.means_)
means.columns=wine.columns[1:]
means

In [ ]:
confusion(wine.Wine_type, qda_model.predict(wine.loc[:,'Alcohol':'Proline']), 
          classes=wine_classes)

***

## Example 2: The Naïve Bayes classifier

Naive Bayes model will assume assume that the attributes of the class conditional probabilities are independent and a certain distribution for them bassed on the kind of data we are working with. 

For example: if we are working with numerical variables it will assume that the features are conditionally independent between them and that they follow a gaussian distribution.

### Naive Bayes on binary data

 Naive Bayes Classifier for Discrete Predictors: we use the 
 1984 United States Congressional Voting Records; 

 This data set includes votes for each of the U.S. House of Representatives Congressmen on 16 key votes
In origin they were nine different types of votes: 
     
* voted for, paired for, and announced for (these three simplified to yea or 'y'),
* voted against, paired against, and announced against (these three simplified to nay or 'n'), 
* voted present, voted present to avoid conflict of interest, and did not vote or otherwise make a position known 
     (these three simplified to an 'unknown' disposition)

 The goal is to classify Congressmen as Republican or Democrat as a function of their voting profiles,
which is not immediate because in the US Congressmen have a large freedom of vote 
 (obviously linked to their party but also to their own feelings, interests and compromises with voters)

In [ ]:
HouseVotes84 = read_csv("house-votes-84.data", 
                        delimiter=',', 
                        header=None,na_values='?')
house_classes = ['democrat','republican']

add meaningful names to the votes

In [ ]:
HouseVotes84.columns=["Class","handicapped_infants","water_project_sharing",
                      "budget_resolution","physician_fee_freeze",
                      "el_salvador_aid","religious_groups_in_schools",
                      "anti_satellite_ban", "aid_to_nicaraguan_contras",
                      "mx_missile","immigration","synfuels_cutback",
                      "education_spending","superfund","crime",
                      "duty_free_exports","export_South_Africa"]
HouseVotes84.describe()

In [ ]:
HouseVotes84.isna().sum()

In [ ]:
for v in HouseVotes84.columns:
    HouseVotes84[v].value_counts(dropna=False)

1 = democrat, 0 = republican
 Note "unknown dispositions" have been treated as missing values!

The naive bayes implementations of scikit-learn do not allow missing values and also need binary data, so we will preprocess first changing *y* for 1 and *n* for 0 and then we perform missing data imputation. Another option would be to eliminate all rows with missing, but that will discard half of the data

In [ ]:
HouseVotes84.replace({'y':1, 'n':0},inplace=True)
HouseVotes84.head()

We use the most frequent value from each column for imputation

In [ ]:
HouseVotes84.loc[:,'handicapped_infants':] =  SimpleImputer(strategy='most_frequent').\
                fit_transform(HouseVotes84.loc[:,'handicapped_infants':])
HouseVotes84.head()

In [ ]:
np.random.seed(1111)
N = HouseVotes84.shape[0]

 We first split the available data into learning and test sets, selecting randomly 2/3 and 1/3 of the data.
 
 We do this for a honest estimation of prediction performance

In [ ]:
train, test = train_test_split(HouseVotes84, test_size=N//3)

We use the BernoulliNB estimator because we have binary data

In [ ]:
model = BernoulliNB().fit(train.loc[:,'handicapped_infants':], 
                          train.Class)

To obtain the probabiblities from the model is a little bit tricky. 

The attribute `class_log_prior_` stores the priot log probabilities for the classes, so we can compute the probabilities doing:

In [ ]:
np.e**model.class_log_prior_

For the attributes/class probabilities is trickier because ony one of the probabilities is stored (the othe is the complement) and also are the log probabilities

In [ ]:
probs=pd.DataFrame({'Democrat Y':np.e**model.feature_log_prob_.T[:,0],
                    'Democrat N':1-np.e**model.feature_log_prob_.T[:,0],
                    'Republican Y':np.e**model.feature_log_prob_.T[:,1], 
                    'Republican N':1-np.e**model.feature_log_prob_.T[:,1]},
                    index=HouseVotes84.columns[1:])

probs

predict the outcome of the first 20 Congressmen

In [ ]:
model.predict(HouseVotes84.loc[0:20,'handicapped_infants':])

In [ ]:
pred=pd.DataFrame(model.predict_proba(HouseVotes84.loc[0:20,'handicapped_infants':]))

pred.columns=['democrat', 'republican']
pred

form and display confusion matrix & overall error

In [ ]:
confusion(train.Class, model.predict(train.loc[:,'handicapped_infants':]), 
          classes=house_classes)

(1-accuracy_score(train.Class, 
                  model.predict(train.loc[:,'handicapped_infants':])))*100

compute the test (prediction) error

In [ ]:
confusion(test.Class,
          model.predict(test.loc[:,'handicapped_infants':]), 
          classes=house_classes)

(1-accuracy_score(test.Class,
                  model.predict(test.loc[:,'handicapped_infants':])))*100

 note how most errors (10/12) correspond to democrats wrongly predicted as republicans

in the event of **empty empirical probabilities**, there is an alpha parameter (0-1) that can be use for performing Laplace correction (aka smoothing) (0 = no smoothing)

In [ ]:
model = BernoulliNB(alpha=0.6).fit(train.loc[:,'handicapped_infants':], 
                                   train.Class)

### Naive Bayes on mixed data types

Now we are going to work with mixed data types using the heart dataset (https://www.kaggle.com/ronitf/heart-disease-uci). 

Our goal now is predict if a patient has a heart disease using medical data. 


In [ ]:
income = pd.read_csv('census_income_weka_dataset.csv')

categorical = ['workclass', 'education', 'marital_status', 'occupation',
       'relationship', 'race', 'sex', 'native_country']
numerical = ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss',
       'hours_per_week']

target= 'income_level'

for c in categorical:
    income[c]=income[c].astype(CategoricalDtype())

In [ ]:
for c in income.columns:
    if c in categorical:
        sn.countplot(x=c,data=income,hue='income_level')
        a =plt.xticks(rotation= 90)
    else:
        sn.histplot(x=c,data=income,hue='income_level')
    plt.show()

As this model assumes independence between features, we can split the dataset by data type and apply two different Naive Bayes models. One Gaussian for the numerical variables and one Categorical for the categorical ones.

In [ ]:
def preprocessing_categorical(X):
    for c in X.columns:
        X[c]=X[c].cat.codes
    return X

We split into train, val and test because we want to compare the results of the different models.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(income[categorical + numerical],income[target],  test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)


X_train_numerical = X_train[numerical]
X_train_categorical = X_train[categorical]
X_train_categorical = preprocessing_categorical(X_train_categorical)

X_val_numerical = X_val[numerical]
X_val_categorical = X_val[categorical]
X_val_categorical = preprocessing_categorical(X_val_categorical)

X_test_numerical = X_test[numerical]
X_test_categorical = X_test[categorical]
X_test_categorical = preprocessing_categorical(X_test_categorical)

Now we train the Gaussian model with the numerical variables and the categorical with the categorical one.

In [ ]:
gaussian_nb = GaussianNB()

gaussian_nb.fit(X_train_numerical,y_train)

gaussian_nb.score(X_val_numerical,y_val)

In [ ]:
cat_nb = CategoricalNB()
cat_nb.fit(X_train_categorical,y_train)

cat_nb.score(X_val_categorical,y_val)

Now we can multiply the probabilities to classify.

In [ ]:
combined_prediction_proba= cat_nb.predict_proba(X_val_categorical) * gaussian_nb.predict_proba(X_val_numerical)

combined_prediction = np.argmax(combined_prediction_proba,axis=1)
clases = {0:'<=50K',1:'>50K'}
combined_prediction = [clases[v] for v in combined_prediction]
accuracy_score(combined_prediction, y_val)

This way we obtain a better validation accuracy. 
Now we are going to analyze the generalization error on the test data.

In [ ]:
test_combined_prediction_proba= cat_nb.predict_proba(X_test_categorical) * gaussian_nb.predict_proba(X_test_numerical)

test_combined_prediction = np.argmax(test_combined_prediction_proba,axis=1)
clases = {0:'<=50K',1:'>50K'}
test_combined_prediction = [clases[v] for v in test_combined_prediction]
accuracy_score(test_combined_prediction, y_test)

In [ ]:
confusion(test_combined_prediction, y_test,['<=50K','>50K'])

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test=='<=50K',  test_combined_prediction_proba[:,0])
auc = metrics.roc_auc_score(y_test=='<=50K',  test_combined_prediction_proba[:,0])
plt.plot(fpr,tpr,label="<=50K, auc="+str(auc))

fpr, tpr, _ = metrics.roc_curve(y_test=='>50K',  test_combined_prediction_proba[:,1])
auc = metrics.roc_auc_score(y_test=='>50K',  test_combined_prediction_proba[:,1])
plt.plot(fpr,tpr,label=">50K, auc="+str(auc))

plt.legend(loc=4)
plt.ylabel('True positive rate')
plt.xlabel('False positive rate')

plt.show()

***

***

## Example 3: Logistic Regression using artificial data

The goal of this example is to get acquainted with the call to glm()
 glm() is used to fit generalized linear models (of which both linear and logistic regression are particular cases)

 You may need to recall at this point the logistic regression model ...

 Let $x$ represent a single continuous predictor
 
 Let $y$ represent a class ('0' or '1'), with a probability of being 1 that is related linearly to the predictor
 via the logit funtion, that is $logit(p) = a*x + b$ (or $beta_1*x + beta_0$ if you prefer)

In [ ]:
np.random.seed(1968)

N = 4000
x = normal (3,2,N)  # generate the x_n 

a = 0.6 
b = -1.5 # this is the ground truth, which is unknown

p = 1/(1+np.exp( -(a*x + b) )) # generate the p_n 
t = binomial(1,p, N)  # generate the targets according to p
data = pd.DataFrame({'x':x, 't':t})

data.plot.scatter('x','t',figsize=(8,8));

In [ ]:
model = GLM.from_formula('t ~ x', data, family=Binomial())
result = model.fit()
result.summary()

 Obviously x is very significant (and the Intercept is always significant)

Therefore, our estimated model is
 $logit(p_n) ={{result.params[1]}}*x_n {{result.params[0]}}$
 quite close to the ground truth

 In general you get this as:
 
  result.params

Interpretation of the coefficients:
 
- For a 1 unit increase in x, there is an increase in the odds for t by a factor of ...

In [ ]:
result.params

In [ ]:
np.exp(result.params[1])

 that is almost doubling the odds (~82% more)

***

## Example 4: Logistic regression for classifying spam mail

 This example will also illustrate how to change the 'cut point' for prediction, when there is an 
 interest in minimizing a particular source of errors

In [ ]:
spam = read_csv("spambase.data", delimiter=',', header=None)
file = open('spambase.names', 'r')
spam.columns = [n.strip() for n in file.readlines()]

spam.head()

We do some basic pre-processing

In [ ]:
spam.loc[:,'capital_run_length_average':'capital_run_length_total'] =\
        spam.loc[:,'capital_run_length_average':'capital_run_length_total'].\
                    apply(lambda x: np.log10(x+1))
spam = spam[spam.word_freq_george==0]
spam = spam[spam.word_freq_650==0]
spam = spam[spam.word_freq_hp==0]
spam = spam[spam.word_freq_hpl==0]
spam =spam.drop(columns=['word_freq_george','word_freq_650',
                         'word_freq_hp','word_freq_hpl'])
spam['about_money']=spam.word_freq_free+spam.word_freq_business+\
spam.word_freq_credit+spam.word_freq_money
spam=spam.drop(columns=['word_freq_free','word_freq_business',
                        'word_freq_credit','word_freq_money'])
Class = spam.Class   # move the Class column to the last position
spam=spam.drop(columns=['Class'])
spam['Class'] = Class

spam.shape


In [ ]:
np.random.seed(4321)
train, test = train_test_split(spam, test_size=0.33)


Fit a GLM in the learning data

In [ ]:
spamM1 = GLM(train.Class,
             add_constant(train.loc[:,:'about_money']), 
             family=Binomial())
resultM1 = spamM1.fit()
resultM1.summary()

We can see that there are some variables that have small weights and are probably not very relevant. The R notebook uses stepwise variable selection to simplify the model.

Statsmodels does not have stepwise variable selection, but we can use crossvalidated Recursive Forward Elimination (RFE) with the implementation of logistic regression from scikit learn. RFE does the same thing as stepwise variable selection but uses accuracy to select the best model using cross validation. The implentation of logistic regression in scikit-learn is more sofisticated and uses regularization so the results will be different than in the R notebook.

In [ ]:
# we use L1 regularization to make 0 a large number 
# of the weigths, the lower the C the more attributes will be discarded
logreg = LogisticRegression(solver='liblinear',penalty='l1',C=1)
#njobs = -1 means that all the cores from the CPU are used
rfe = RFECV(estimator=logreg,cv=10,n_jobs=-1) 
rfe.fit(train.loc[:,:'about_money'],train.Class);

In [ ]:
print('Features Selected:',rfe.n_features_)
print('\n Ranking of features')
sel = pd.DataFrame({'features': train.columns[:-1],
                    'ranking': rfe.ranking_, 
                    'selected':rfe.support_})
sel.sort_values(by='ranking')

We get the extimator from the RFE and the list of selected variable to slice the data matrix

In [ ]:
resultM1 = rfe.estimator_
sel_features = list(sel.features[sel.selected])
sel_features


 We define now a convenience function:

 'P' is a parameter; whenever our filter assigns spam with probability at least P then we predict spam

In [ ]:
def spam_acc(P=0.5):
    # We use predict_proba instead of prediction to obtain 
    # the probabilities of the classes and 
    # we select only the probability for class 1 as 
    # the other is just the complementary
    
    # Accuracy in training
    pred = resultM1.predict_proba(train.loc[:,sel_features])[:,1]
    lab_tr = [1 if i>=P else 0 for i in pred]
    df_tr=confusion(train.Class,lab_tr, classes=['nospam','spam'])

    # Accuracy in test
    pred = resultM1.predict_proba(test.loc[:,sel_features])[:,1]
    lab_ts = [1 if i>=P else 0 for i in pred]
    df_ts=confusion(test.Class,lab_ts, classes=['nospam','spam'])
 
    return df_tr, (1-accuracy_score(train.Class,lab_tr))*100,\
           df_ts, (1-accuracy_score(test.Class,lab_ts))*100

In [ ]:
c_tr,e_tr,c_ts,e_ts= spam_acc()
c_tr
print(f'Training error: {e_tr}%')
c_ts
print(f'Test error: {e_ts}%')

 Although the errors are quite low still one could argue that we should try to lower the probability of predicting spam when it is not
 We can do this (at the expense of increasing the converse probability) by:

In [ ]:
c_tr,e_tr,c_ts,e_ts= spam_acc(0.7)
c_tr
print(f'Training error: {e_tr}%')
c_ts
print(f'Test error: {e_ts}%')

 So we get a much better spam filter; notice that the filter has a very low probability of 
predicting spam when it is not (which is the delicate case), of about 

In [ ]:
c_ts.loc['nospam','spam'] /c_ts.loc['nospam'].sum()*100